In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [20]:
# Import data
weather_data_df = pd.read_csv('../WeatherPy/output_data/weather_df.csv')
weather_data_df

,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.9266,125.4092,60.26,90,91,3.13,TL,1651329814
1,Kirakira,-10.4544,161.9205,79.93,81,100,0.58,SB,1651329897
2,Port Alfred,-33.5906,26.8910,67.17,66,2,7.00,ZA,1651329657
3,Sarakhs,36.5449,61.1577,96.94,19,75,4.61,IR,1651329897
4,Vaini,-21.2000,-175.2000,77.16,94,100,10.36,TO,1651329801
...,...,...,...,...,...,...,...,...,...
562,Flin Flon,54.7682,-101.8650,40.84,75,75,4.61,CA,1651329871
563,Krasnoarmeyskoye,52.7182,50.0322,55.17,53,73,6.20,RU,1651330146
564,Rurrenabaque,-14.4413,-67.5278,81.23,65,85,3.67,BO,1651330147
565,Vestmannaeyjar,63.4427,-20.2734,47.39,76,100,16.11,IS,1651329903


 ### Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [21]:
# Configure Google Maps API
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data_df[['Lat', 'Lng']]

# Humidity
humidity = weather_data_df['Humidity']

# Heat map plot
fig = gmaps.figure(center=(0, 0), zoom_level=2)
# Create heat map layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=10)

# Add heatmap layer to map
fig.add_layer(heatmap_layer)
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [22]:
# Narrow down the cities to fit weather conditions
ideal_temp = weather_data_df[(weather_data_df['Maxtemp'] < 80)&  (weather_data_df['Maxtemp'] >70)]
ideal_cloudiness = ideal_temp[ideal_temp['Cloudiness'] == 0]
ideal_windspeed = ideal_cloudiness[ideal_cloudiness['Wind Speed'] < 10]

# Drop the rows with null values and index column after resetting the index
ideal_vacation_spots = ideal_windspeed[ideal_windspeed['Humidity'] < 60].dropna()

ideal_vacation_spots = ideal_vacation_spots[['City','Lat','Lng','Maxtemp','Humidity','Cloudiness','Wind Speed','Country','Date']]
ideal_vacation_spots.reset_index(inplace=True, drop=True) 
ideal_vacation_spots

,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Evans,33.5337,-82.1307,75.25,56,0,8.05,US,1651329929
1,Ponta do Sol,32.6667,-17.1000,72.23,49,0,0.96,PT,1651329818
2,Zhezkazgan,47.8043,67.7144,71.58,37,0,4.47,KZ,1651329968
3,Ojinaga,29.5667,-104.4167,79.57,15,0,3.44,MX,1651329912
4,Brawley,32.9787,-115.5303,76.60,36,0,3.44,US,1651330124
5,Redlands,34.0556,-117.1825,77.94,50,0,0.00,US,1651330136


### Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [23]:
# Store into variable named hotel_df
hotel_df = ideal_vacation_spots
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = " "
# Set parameters to search for hotels with 5000 meters.
for index,city in hotel_df.iterrows():
    hotel_lat = city['Lat']
    hotel_lng = city['Lng']
    hotel_coordinates = f"{hotel_lat},{hotel_lng}"
    params = {
        'location': hotel_coordinates,
        'radius': 5000,
        'types': 'lodging',
        'key': g_key
    }
    
    # Send the request to Google Maps.
    hotel_response = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params=params)
    hotel_names = hotel_response.json()
        
    # Extract the hotel names from the response.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_names['results'][0]['name']
    except:
        print("No hotels found")
hotel_df 

,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Evans,33.5337,-82.1307,75.25,56,0,8.05,US,1651329929,Evans hide out
1,Ponta do Sol,32.6667,-17.1000,72.23,49,0,0.96,PT,1651329818,Hotel do Campo
2,Zhezkazgan,47.8043,67.7144,71.58,37,0,4.47,KZ,1651329968,Hotel Zumrat
3,Ojinaga,29.5667,-104.4167,79.57,15,0,3.44,MX,1651329912,Suites Las Palmas
4,Brawley,32.9787,-115.5303,76.60,36,0,3.44,US,1651330124,Brawley Inn Hotel & Conference Center
5,Redlands,34.0556,-117.1825,77.94,50,0,0.00,US,1651330136,Dynasty Suites Redlands


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add the hotel marks to the map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))